In [77]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 
from torch import tensor 
import numpy as np

torch.set_warn_always(False)

In [78]:
dir(torch.autograd.Function)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_backward_cls',
 '_compiled_autograd_backward_state',
 '_compiled_autograd_key',
 '_get_compiled_autograd_symints',
 '_is_compiled_autograd_tracing',
 '_materialize_non_diff_grads',
 '_raw_saved_tensors',
 '_register_hook',
 '_register_hook_dict',
 '_sequence_nr',
 '_set_sequence_nr',
 'apply',
 'backward',
 'dirty_tensors',
 'forward',
 'generate_vmap_rule',
 'is_traceable',
 'jvp',
 'mark_dirty',
 'mark_non_differentiable',
 'mark_shared_storage',
 'materialize_grads',
 'maybe_clear_saved_tensors',
 'metadata',
 'name',
 'needs_input_grad',
 'next_functions',
 'non_differentiable',
 'regis

we can set `requires_grad=True` for the weights we want to calculate gradients for. usually we set this for leaf nodes(w, b)

if x has `requires_grad=True` ==> y = f(x) also has `requires_grad=True`

In [79]:
x = torch.tensor(1.0, requires_grad=True, dtype=torch.float64)
y = torch.sigmoid(x)
z = 2*y
print(z.requires_grad)
print(y.grad_fn)

True


In [80]:
round(1/(1+np.exp(-x.item())), 6) == round(y.item(), 6)

True

In [81]:
print(x.data)
print(x.grad)
print(x.grad_fn)
print(x.requires_grad)
print(x.backward)

tensor(1., dtype=torch.float64)
None
None
True
<bound method Tensor.backward of tensor(1., dtype=torch.float64, requires_grad=True)>


if we make `requires_grad=False` in x then y also becomes a leaf node

In [82]:
print(x.is_leaf, y.is_leaf, y)

True False tensor(0.7311, dtype=torch.float64, grad_fn=<SigmoidBackward0>)


In [83]:
y.backward()

In [84]:
print(x.grad)

tensor(0.1966, dtype=torch.float64)


In [91]:
y.item()*(1-y.item()) #gradient of sigmoid 

0.19661193324148185